# Explore here

It's recommended to use this notebook for exploration purposes.

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",  # Pretend to be a browser
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"
}

In [3]:
# Fetch the webpage

url = 'https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records'
response = requests.get(url,headers=headers) 

In [4]:
print("RESPONSE STATUS IS: ", response.status_code)

RESPONSE STATUS IS:  200


In [5]:
# Extract tables with pandas
html = io.StringIO(response.text)  # Convert the HTML to a text file

# read_html() returns a list of DataFrames
tables = pd.read_html(html) # Scans for <table> tags and converts it into pandas DataFrame. Built to only parse for <table> default behavior
print(f"{len(tables)} tables were found.") # Checks the length of "tables" variable

27 tables were found.


In [6]:
df = tables[0]
df.drop("Ref.", axis=1, inplace=True)
df


,Rank,Song,Artist(s),Streams (billions),Release date
0,1,"""Blinding Lights""",The Weeknd,5.039,29 November 2019
1,2,"""Shape of You""",Ed Sheeran,4.562,6 January 2017
2,3,"""Starboy""",The Weeknd and Daft Punk,4.105,21 September 2016
3,4,"""Someone You Loved""",Lewis Capaldi,4.061,8 November 2018
4,5,"""As It Was""",Harry Styles,4.047,1 April 2022
...,...,...,...,...,...
96,97,"""Jocelyn Flores""",XXXTentacion,2.416,31 October 2017
97,98,"""Sweet Child O' Mine""",Guns N' Roses,2.415,3 June 1988
98,99,"""Clean Baby Sleep White Noise (Loopable)""",Dream Supplier,2.407,29 April 2020
99,100,"""Someone Like You""",Adele,2.405,24 January 2011


In [7]:
df = df.drop(df.index[-1])


In [8]:


# --- Create empty database instance ---
conn = sqlite3.connect("songs.db")  # creates a new SQLite database file
cursor = conn.cursor()

# --- Create the table ---
cursor.execute("""
CREATE TABLE IF NOT EXISTS songs (
    Rank INTEGER PRIMARY KEY,
    Song TEXT,
    Artist TEXT,
    Streams REAL,
    ReleaseDate TEXT
)
""")

# --- Insert values ---
for _, row in df.iterrows():
    cursor.execute("""
    INSERT INTO songs (Rank, Song, Artist, Streams, ReleaseDate)
    VALUES (?, ?, ?, ?, ?)
    """, (row["Rank"], row["Song"], row["Artist(s)"], row["Streams (billions)"], row["Release date"]))

# --- Commit changes and close ---
conn.commit()
conn.close()

print("Database created and data inserted successfully!")


Database created and data inserted successfully!
